In [122]:
import os

PROJECT_ID = "swift-kiln-380818"
LOCATION = "us-central1"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/aditygrg/.config/gcloud/application_default_credentials.json"

import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession

vertexai.init(location=LOCATION, project=PROJECT_ID)

In [123]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    SafetySetting,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    Tool,
)
from vertexai.preview import rag

In [224]:
get_data_of_user = FunctionDeclaration(
    name = "get_data_of_user",
    description = 
    """
        Do not mock up any info, ask here if you need any info.
        Get data of User, his orders, product, transactions, items info and any other profile details.
    """,
    parameters = {
        "type": "object",
        "properties": {"query": {"type": "string"}},
    }
)

get_answers_to_general_help = FunctionDeclaration(
    name = "get_answers_to_general_help",
    description = "Get answers for everything about Amazon, but not user related. It contains information of all Amazon Help docs.",
    parameters = {
        "type": "object",
        "properties": {"query": {"type": "string"}},
    }
)

terminate_if_satisfied = FunctionDeclaration(
    name = "terminate_if_satisfied",
    description = "Checks if the user seems satisfied with the answer and terminates the chat accordingly.",
    parameters = {
        "type": "object",
        "properties": {"feedback_user": {"type": "string"}},
    }
)

send_to_agent_for_manual_intervention = FunctionDeclaration(
    name = "send_to_agent_for_manual_intervention",
    description = "Sends the query to a human agent for manual intervention when the LLM is unable to process it.",
    parameters = {
        "type": "object",
        "properties": {"query": {"type": "string"}},
    }
)

In [225]:
#Implementation 1

retail_tool = Tool(
    function_declarations=[
        get_data_of_user,
        get_answers_to_general_help,
        terminate_if_satisfied,
        send_to_agent_for_manual_intervention
    ],
)

In [226]:
SYSTEM_INSTRUCTION = """
    Your name is Radhika from Amazon Customer Support Agent Team.

    You are on a call. 
    
    You can use any of the tools for help 
    get_data_of_user,
    get_answers_to_general_help,
    terminate_if_satisfied,
    send_to_agent_for_manual_intervention

    Do not verify anything about user because he is already verified.

    You can ask user if you need any details.
    
    If you are unable to find the answer, forward the call to agent by calling 'send_to_agent_for_manual_intervention'

    You can only tell the user about things, you cannot help if any thing needs to be executed. 
    For example, you cannot execute an return but you can only tell info about it.

    DO NOT MAKE UP ANSWERS/DETAILS OF YOUR OWN.
"""

In [227]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0.5),
    tools=[retail_tool],
    system_instruction=SYSTEM_INSTRUCTION,
)
chat = model.start_chat(response_validation=False)

In [228]:
response = chat.send_message("Hi, I accidentally placed an order for the wrong item. Is there any way to cancel it before it ships?")
response.candidates[0].content.parts[0]

text: "Hello! I understand you are calling to cancel an order that was placed accidentally, is that correct? \n"

In [229]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Hello! I understand you are calling to cancel an order that was placed accidentally, is that correct? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0766838044
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0664479285
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0468120314
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0904137269
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.210441604
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0940101817
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0497734509
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.056

In [221]:
response = chat.send_message("2345675432")

In [222]:
function_called_response = response

In [223]:
function_called_response

candidates {
  content {
    role: "model"
    parts {
      text: "Hello! I understand you are calling to cancel an order placed accidentally, is that right? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0476912819
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0490396284
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0652465671
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.08464396
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.157784641
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0793528929
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0543993153
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.053601142
  }
}


In [200]:
response = chat.send_message(
    Part.from_function_response(
        name="get_order_details",
        response={
            "content": "I couldnt find this order detail",
        },
    ),
)

In [201]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "I am sorry, I am unable to locate an order with that id. Could you please double check the order id? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0329612382
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0409223028
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.217836261
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0177803896
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0338440351
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0138486102
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0915444046
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0316

In [51]:
safety_config = [
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_UNSPECIFIED,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_HARASSMENT,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                threshold=HarmBlockThreshold.BLOCK_NONE
            )
        ]


In [301]:
messages = [
    ("User", prompt),
    ("AI", "Welcome to Amazon! Hope you are having a good day.\n\nI understand you are facing some difficulties in ordering data. I am more than happy to help!\n\nBefore we proceed, for security purposes, could you please provide me with the pin code associated with your Amazon account?\n"),
    ("Human", "I dont know my pin code"),
]

In [302]:
def convert_to_str(messages):
    ans = ""
    for i in messages:
        ans += i[0] + " : " + i[1] + "\n"

    return ans

In [304]:
res = chat.send_message(convert_to_str(messages), safety_settings=safety_config)

In [305]:
res.candidates[0].content.parts[0]

function_call {
  name: "get_user_data"
  args {
    fields {
      key: "phone_number"
      value {
        string_value: "932489237"
      }
    }
  }
}

In [306]:
messages = [
    ("User", prompt),
    ("AI", "Welcome to Amazon! Hope you are having a good day.\n\nI understand you are facing some difficulties in ordering data. I am more than happy to help!\n\nBefore we proceed, for security purposes, could you please provide me with the pin code associated with your Amazon account?\n"),
    ("Human", "I dont know my pin code"),
    ("AI", "That\'s alright! Can you please provide me with the city and state associated with your Amazon account? \n"),
    ("Human", "My city is Indore")
]

In [307]:
res = chat.send_message(convert_to_str(messages), safety_settings=safety_config)

In [308]:
res

candidates {
  content {
    role: "model"
    parts {
      text: "That\'s alright! Could you please also tell me the state associated with your Amazon account? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0987725928
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.103930376
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.318061382
    severity: HARM_SEVERITY_LOW
    severity_score: 0.341582507
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.107812636
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0872522444
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.105210543
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0699218363
  }
}
usage

In [309]:
messages = [
    ("User", prompt),
    ("AI", "Welcome to Amazon! Hope you are having a good day.\n\nI understand you are facing some difficulties in ordering data. I am more than happy to help!\n\nBefore we proceed, for security purposes, could you please provide me with the pin code associated with your Amazon account?\n"),
    ("Human", "I dont know my pin code"),
    ("AI", "That\'s alright! Can you please provide me with the city and state associated with your Amazon account? \n"),
    ("Human", "My city is Indore"),
    ("AI", "That\'s alright! Could you please also tell me the state associated with your Amazon account? \n"),
    ("Human", "Madhya Pradesh")
]

In [310]:
res = chat.send_message(convert_to_str(messages), safety_settings=safety_config)

In [311]:
res

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_user_data"
        args {
          fields {
            key: "phone_number"
            value {
              string_value: "932489237"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.192084283
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.114963353
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: MEDIUM
    probability_score: 0.751648128
    severity: HARM_SEVERITY_MEDIUM
    severity_score: 0.544619739
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.169131249
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.134094119
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probab

In [ ]:
response_confirm = res

In [ ]:
print(not response_confirm.candidates[0].content.parts[0].function_call)

True


In [312]:
res.candidates[0]

content {
  role: "model"
  parts {
    function_call {
      name: "get_user_data"
      args {
        fields {
          key: "phone_number"
          value {
            string_value: "932489237"
          }
        }
      }
    }
  }
}
finish_reason: STOP
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
  probability_score: 0.192084283
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.114963353
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: MEDIUM
  probability_score: 0.751648128
  severity: HARM_SEVERITY_MEDIUM
  severity_score: 0.544619739
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
  probability_score: 0.169131249
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.134094119
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
  probability_score: 0.207857162
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.0726367
}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="user_not_verified",
        response={
            "content": "Sorry, we were unable to verify your details. Thank you for contacting Amazon!",
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Sorry, we were unable to verify your details. Thank you for contacting Amazon! \n"

In [ ]:
prompt = """
What about the Pixel 8? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

In [ ]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

In [ ]:
api_response = {"sku": "GA08475-US", "in_stock": "yes"}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

In [ ]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

In [ ]:
api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

In [ ]:

response.candidates[0].content.parts[0]

In [368]:
from mongoengine import connect
from dotenv import load_dotenv
import os
import models.address as Address, models.orders as Order, models.product as Product, models.transactionDetail as Transaction, models.user as User
import json as JSON
from bson.json_util import dumps

load_dotenv()

client = connect(host=os.environ['MONGO_URL'])

all_users = client.list_database_names()
print(all_users)

['amazon_data', 'admin', 'local']


In [369]:
db = client['amazon_data']

In [394]:
result

[]